In [1]:
from pandas_datareader import data as pdr 
from datetime import date
import yfinance as yf 
yf.pdr_override()
import pandas as pd
import matplotlib.pyplot as plt 
import math
import quandl
import numpy as np
from sklearn import preprocessing
%matplotlib qt

c:\users\juand\appdata\local\programs\python\python36\lib\site-packages\pandas_datareader\compat\__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [14]:
ticker_sp = '^GSPC'
ticker_gold = 'GC=F'
ticker_oil = 'CL=F'
ticker_dax = '^GDAXI'
ticker_nikkei = '^N225'
ticker_ftse = '^FTSE'
ticker_shanghai = '000001.SS'

auth_tok = "Nv1rJgRR7u88iz_dg7Y6"

end_date = "2020-07-1"
start_date = "1970-01-02"

In [15]:
def getGOLDData ():
    # Contains only price from 1975 onwards
    data = quandl.get("CHRIS/CME_GC1", trim_start = start_date, trim_end = end_date, authtoken=auth_tok)
    data = data[['Last']]
    data.columns = ["GOLD Adj Close"]
    return data

def getSPData():
    # Contains price from 1970 onwards
    data = pdr.get_data_yahoo(ticker_sp, start=start_date, end=end_date)
    data = data[data.columns[4:5]] 
    data.columns = ["SP500 Adj Close"]
    return data

def getDAXData():
    # Contains price from 1988 onwards
    data = pdr.get_data_yahoo(ticker_dax, start=start_date, end=end_date)
    data = data[data.columns[4:5]]
    data.columns = ["DAX Adj Close"]
    return data


def getOILData():
    # Contains only price FROM 1984 onwards
    data = quandl.get("CHRIS/CME_CL1", trim_start = start_date, trim_end = end_date, authtoken=auth_tok)
    data = data[["Last"]]
    data.columns=["OIL Adj Close"]
    return data


def getNIKKEIData():
    # Contains only price from 1970 onwards
    data = pdr.get_data_yahoo(ticker_nikkei, start=start_date, end=end_date)
    data = data[data.columns[4:5]] 
    data.columns = ["NIKKEI Adj Close"]
    return data


def getFTSEData():
    # Contains price from 1984 onwards
    data = pdr.get_data_yahoo(ticker_ftse, start=start_date, end=end_date)
    data = data[data.columns[4:5]] 
    data.columns = ["FTSE Adj Close"]
    return data

def getSHANGHAIData():
    # Contains only price from 1997
    data = pdr.get_data_yahoo(ticker_shanghai, start=start_date, end=end_date)
    data = data[data.columns[4:5]] 
    data.columns = ["SHANGHAI Adj Close"]
    return data

In [25]:
plt.plot(getSHANGHAIData())
plt.show()

[*********************100%***********************]  1 of 1 completed


In [26]:
def combineData():
    allData = [getSPData(), getDAXData(), getFTSEData(), getGOLDData(), getOILData()]
    mergedData = pd.concat(allData, axis = 1)
    cleanData = mergedData.dropna()
    return cleanData

In [82]:
data = combineData()
print(data)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
            SP500 Adj Close  DAX Adj Close  FTSE Adj Close  GOLD Adj Close  \
Date                                                                         
1987-12-30       247.860001    1005.190002     1759.800049           485.5   
1988-01-04       255.940002     956.489990     1713.900024           480.5   
1988-01-05       258.630005     996.099976     1789.599976           483.2   
1988-01-06       258.890015    1006.010010     1787.099976           485.3   
1988-01-07       261.070007    1014.469971     1787.199951           483.1   
...                     ...            ...             ...             ...   
2020-06-04      3112.350098   12430.559570     6341.399902          1712.3   
2020-06-05      3193.929932   12847.679688     6484.299805          1679.5   
2020-06-08   

In [83]:
FUTURE_TO_PREDICT = 1

data['Future'] = data['SP500 Adj Close'].shift(-FUTURE_TO_PREDICT)

data.dropna(inplace=True)

print(data)

            SP500 Adj Close  DAX Adj Close  FTSE Adj Close  GOLD Adj Close  \
Date                                                                         
1987-12-30       247.860001    1005.190002     1759.800049           485.5   
1988-01-04       255.940002     956.489990     1713.900024           480.5   
1988-01-05       258.630005     996.099976     1789.599976           483.2   
1988-01-06       258.890015    1006.010010     1787.099976           485.3   
1988-01-07       261.070007    1014.469971     1787.199951           483.1   
...                     ...            ...             ...             ...   
2020-06-03      3122.870117   12487.360352     6382.399902          1694.0   
2020-06-04      3112.350098   12430.559570     6341.399902          1712.3   
2020-06-05      3193.929932   12847.679688     6484.299805          1679.5   
2020-06-08      3232.389893   12819.589844     6472.600098          1698.9   
2020-06-09      3207.179932   12617.990234     6335.700195      

In [84]:
def buy_or_sell (current, future):
    if (future > current):
        return 1
    else:
        return 0
    
data['Target'] = list(map(buy_or_sell, data['SP500 Adj Close'], data['Future']))

In [85]:
NUMBER_OF_DATA_POINTS = len(data)
SIZE_TRAINING = int(NUMBER_OF_DATA_POINTS * 0.85)
SIZE_TESTING  = NUMBER_OF_DATA_POINTS - SIZE_TRAINING
print("size of training data: {}".format(SIZE_TRAINING))
print("size of testing data: {}".format(SIZE_TESTING))

data_training = data[:SIZE_TRAINING]
data_testing  = data[SIZE_TRAINING:]

size of training data: 6676
size of testing data: 1179


In [86]:
print(data_training)

            SP500 Adj Close  DAX Adj Close  FTSE Adj Close  GOLD Adj Close  \
Date                                                                         
1987-12-30       247.860001    1005.190002     1759.800049           485.5   
1988-01-04       255.940002     956.489990     1713.900024           480.5   
1988-01-05       258.630005     996.099976     1789.599976           483.2   
1988-01-06       258.890015    1006.010010     1787.099976           485.3   
1988-01-07       261.070007    1014.469971     1787.199951           483.1   
...                     ...            ...             ...             ...   
2015-06-18      2121.239990   11100.299805     6707.899902          1201.5   
2015-06-19      2109.989990   11040.099609     6710.500000          1201.5   
2015-06-22      2122.850098   11460.500000     6825.700195          1183.7   
2015-06-23      2124.199951   11542.540039     6834.899902          1176.2   
2015-06-24      2108.580078   11471.259766     6844.799805      

In [87]:
def process_data(data):
    data.drop("Future", axis = 1, inplace = True)  # Drop the future column so that the NN doesn't have access to the future
    
    for column in data.columns:  # Normalize the columns
        if column != "Target":   # We only want to normalize the other columns
            data[column] = data[column].pct_change()   # Normalization by percent change
            data.dropna(inplace = True)
            #data[column] = preprocessing.scale(data[column].values)    # Scale values between 0 and 1
    data.dropna(inplace = True)
    
    sequential_data = []
    sequence_length = 15
    
    for day in range (len(data)-sequence_length+1):
        
        sequence = []
        
        for future_day in range (sequence_length):
            sequence.append(data.iloc[day + future_day].values[:-1])
        
        buy_or_sell = data.iloc[day + sequence_length - 1].values[-1]
        
        sequential_data.append(sequence, buy_or_sell)
        
    random.shuffle(sequential_data)
    
    buy_sequences  = []
    sell_sequences = []
    
    for sequence, target in sequential_data:
        if target == 1:
            buy_sequences.append(sequence)
        elif target == 0:
            sell_sequences.append(sequence)
        
        
    
    return data

In [88]:
data_training = process_data(data_training)
data_testing  = process_data(data_testing)

c:\users\juand\appdata\local\programs\python\python36\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
c:\users\juand\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
c:\users\juand\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/

In [104]:
print(data_training.iloc[0].values)

[-0.06768305  0.01204567 -0.00772153  0.00496792 -0.00460035  1.        ]


In [101]:
print(max(data_testing['GOLD Adj Close'].values))

0.06488917442981057
